# Tutorial Rhizodep Nitrogen

In [1]:
import pickle
from math import exp

import oawidgets
from oawidgets.plantgl import PlantGL, MTG
from oawidgets.mtg import plot

from rhizodep.nitrogen import init_N, transport_N, update_N
from rhizodep.tools import plot_mtg

ModuleNotFoundError: No module named 'oawidgets'

In [2]:
def load_mtg():
    with open('../test/inputs/root00119.pckl', 'rb') as f:
        g = pickle.load(f)
    return g

g = load_mtg()
plot(g)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [3]:
def init_soil(g,
              zmax_soil_Nm:float = -0.02,
              soil_Nm_variance:float = 0.0001,
              soil_Nm_slope:float = 25,
              scenario:int = 0 
             ):
    props = g.properties()
    soil_Nm = props['soil_Nm']
    z1 = props['z1']

    # No order in update propagation
    max_scale = g.max_scale()
    for vid in g.vertices(scale=max_scale):
        # Soil concentration heterogeneity as border conditions

        soil_Nm[vid] = (
                ( 0.01 * exp(-((z1[vid]-zmax_soil_Nm)**2)/soil_Nm_variance) )**(scenario)
                * (1 + soil_Nm_slope * z1[vid])**(1 - scenario)
                        )

    return g


In [4]:
def plot_N(g,
           p: str = 'influx_Nm'):
    props = g.property(p)
    max_scale = g.max_scale()
    plot_range = [props[vid] for vid in g.vertices(scale=max_scale) if props[vid] != 0]

    scene = plot_mtg(g,
                     prop_cmap=p,
                     lognorm=False,  # to avoid issues with negative values
                     vmin=min(plot_range),
                     vmax=max(plot_range),
                     viewer=False
                     )
    return scene

In [5]:
def test_nitrogen_patch(n=10):
    g = load_mtg()

    # Initialization of state variable
    g = init_N(g)
    g = init_soil(g, scenario=1)

    for i in range(n):
        g = transport_N(g)
        g = update_N(g)
        # print_g(g, select, vertice=19)
    #plot_N(g, p='influx_Nm')

    #print_g(g)
    #print(g.node(0).xylem_Nm, g.node(0).xylem_volume)

    return g

In [6]:
g = test_nitrogen_patch()

In [7]:
scene = plot_N(g)

In [11]:
g.properties()['geometry']= { vid: shs[0] for vid, shs in scene.todict().items()}

In [12]:
MTG(g, 'influx_Nm')

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [13]:
PlantGL(scene)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…